In [4]:
import pandas as pd 
import numpy as np
import scipy as sp

# specificity score

In [5]:
outdir= '/home/UTHSCSA/hef/2.Project/z.other/2.Jack_lab/final_2_batch/'

In [6]:
ligands = pd.read_table(outdir + 'ligand_477.txt') 
ligands = list(ligands['ligand genes'])

exp = pd.read_table(outdir + '/data_input.txt', sep = '\t', index_col = 0)
exp_ligand = exp.T[[i for i in ligands if i in list(exp.index)]]
exp_ligand = exp_ligand.T.reset_index().groupby('gene').mean().T 
exp_ligand.index = exp_ligand.index.str.replace(' $', '', regex = True)

celltype = pd.read_table(outdir + '/id_rename.txt', sep = '\t',header = None)
exp_ligand['Type'] = exp_ligand.index.map(dict(celltype.values))
exp_ligand

gene,Adam10,Adcyap1,Adipoq,Adm,Adm2,Agrp,Agt,Ahsg,Alb,Alox5ap,...,Wnt4,Wnt5a,Wnt5b,Wnt7a,Wnt7b,Wnt8a,Wnt8b,Wnt9b,Xcl1,Type
B.nil.1,2816.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,814.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B.nil
B.nil.2,3432.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,1089.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B.nil
B.nil.3,2691.0,0.0,0.0,2.0,0.0,18.0,0.0,0.0,0.0,6415.0,...,12.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,141.0,B.nil
B.nil.4,2894.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6336.0,...,6.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,234.0,B.nil
B.LPS.1,2376.0,0.0,0.0,2.0,0.0,8.0,0.0,0.0,0.0,13185.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,434.0,B.LPS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HSC.MPP.1,4504.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,2802.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HSC.MPP
HSC.MPP.2,3165.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,2.0,2943.0,...,2.0,5.0,5.0,0.0,0.0,0.0,3.0,2.0,0.0,HSC.MPP
HSC.MPP.3,3173.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,10.0,2861.0,...,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,HSC.MPP
HSC.MPP.4,3511.0,2.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,3142.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,HSC.MPP


In [ ]:
import os
os.makedirs(outdir + '/key_scores', exist_ok=True)
cell_types = list(exp_ligand['Type'].unique())

p = []
for item1 in cell_types:
    for item2 in cell_types:
        if item2 != item1:
            for ge in exp_ligand.columns[:-1]:
                a = exp_ligand[exp_ligand['Type'] == item1][ge].to_list()
                b = exp_ligand[exp_ligand['Type'] == item2][ge].to_list()
                p.append([ge, item1, np.mean(a), item2, np.mean(b), 
                          sp.stats.mannwhitneyu(a, b, alternative='greater')[1], sp.stats.ttest_ind(a,b, alternative='greater')[1]])

p = pd.DataFrame(p, columns = ['gene', 'celltype1', 'mean1','celltype2','mean2','pvalue1', 'pvalue2']).fillna(1)
p['fdr1'] = fdr(p, 'pvalue1')
p['fdr2'] = fdr(p, 'pvalue2')
p.to_csv(outdir + '/key_scores/celltype_vs_celltype.pvalue.txt', sep = '\t')

In [8]:
p = pd.read_table(outdir + '/key_scores/celltype_vs_celltype.pvalue.txt', sep = '\t', index_col = 0)
p['log2FC'] = np.log2((p['mean1']+0.001)/(p['mean2']+0.001) +0.001) #pseudocount
p = p.replace([np.inf, -np.inf], np.nan)

p = p[(p['mean1'] > 20) & (p['pvalue1'] < 0.01)  & (p['log2FC'] > 2)]
specificity = p.groupby(['celltype1','gene']).count()[['mean1']].reset_index()
specificity.columns = ['celltype1', 'gene', 'spec_score']
specificity[specificity['celltype1'] == 'B.LC/21'].sort_values('spec_score', ascending =False).head(10)

,celltype1,gene,spec_score
246,B.LC/21,Il27,21
293,B.LC/21,Tnc,17
247,B.LC/21,Il33,16
303,B.LC/21,Vegfa,15
221,B.LC/21,Cxcl10,15
210,B.LC/21,Cd40lg,15
292,B.LC/21,Tigit,14
259,B.LC/21,Madcam1,14
243,B.LC/21,Il1b,13
288,B.LC/21,Spp1,12


In [9]:
n = 10
top = pd.DataFrame()
for ct1 in specificity.celltype1.unique():
    top = pd.concat([top, specificity[specificity['celltype1'] == ct1].sort_values('spec_score', ascending =False).head(n)], axis = 0)
top.to_csv('/home/UTHSCSA/hef/2.Project/z.other/2.Jack_lab/tsne_compare/specScore_top'+str(n)+'/top_gene.mean20_pv0.01_log2fc2.txt', sep = '\t', index = False)
top 

,celltype1,gene,spec_score
39,B.CD154,Madcam1,15
0,B.CD154,Adipoq,13
7,B.CD154,Ccl1,13
35,B.CD154,Lpl,13
51,B.CD154,Raet1d,12
...,...,...,...
4066,T.gd,Xcl1,19
3985,T.gd,Fasl,19
4060,T.gd,Tnfsf8,19
4053,T.gd,Timp2,19


# interaction score

## data prepare

In [13]:
exp = pd.read_table('/home/UTHSCSA/hef/2.Project/z.other/2.Jack_lab/final_2_batch/data_input.txt', index_col=0)
exp = exp.replace(' ', '', regex = True).astype(int)
exp 

,B.nil.1,B.nil.2,B.nil.3,B.nil.4,B.LPS.1,B.LPS.2,B.LPS.3,B.LPS.4,B.CD154.1,B.CD154.2,...,HSC.LTHSC.2,HSC.LTHSC.3,HSC.LTHSC.4,HSC.STHSC.1,HSC.STHSC.2,HSC.MPP.1,HSC.MPP.2,HSC.MPP.3,HSC.MPP.4,HSC.MPP.5
gene,,,,,,,,,,,,,,,,,,,,,
0610030E20Rik,1125,1143,996,934,546,776,758,772,643,789,...,840,1067,835,1274,947,789,786,502,769,668
0610040J01Rik,0,0,112,117,0,0,68,0,0,0,...,32,8,7,6,0,11,6,0,11,0
1110002E22Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,7
1110008P14Rik,429,432,324,332,234,285,311,347,247,228,...,78,92,98,139,100,180,99,121,98,104
1110032A03Rik,287,295,290,278,165,115,139,172,252,295,...,453,365,322,419,486,338,358,308,403,332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd3,18,167,81,114,252,277,181,235,323,317,...,352,324,227,361,265,374,248,298,290,211
mt-Nd4,21934,10470,11101,12559,11278,13841,12076,12723,19510,17866,...,16263,16167,15582,17367,18704,16809,16013,16754,15880,15080
mt-Nd4l,14,0,2,0,5,1,1,1,6,1,...,0,0,8,0,0,0,0,0,0,0


In [20]:
pair = pd.read_table('/home/UTHSCSA/hef/2.Project/z.other/2.Jack_lab/final_2_batch/pair_table.txt', sep = '\t')


input_dir = '/home/UTHSCSA/hef/2.Project/z.other/2.Jack_lab/tsne_compare/specScore_top10'
top_gene = pd.read_table(input_dir + '/top_gene.mean20_pv0.01_log2fc2.txt')
gene_lst = top_gene.gene.unique() 
pair_sel = pair.set_index('GeneA').T[gene_lst].T.reset_index() 

exp_sel = exp.T[list(set(list(pair_sel.GeneA) + list(pair_sel.GeneB)))].T 
exp_sel.to_csv(input_dir + '/exp.txt', sep = '\t')
exp_sel

,B.nil.1,B.nil.2,B.nil.3,B.nil.4,B.LPS.1,B.LPS.2,B.LPS.3,B.LPS.4,B.CD154.1,B.CD154.2,...,HSC.LTHSC.2,HSC.LTHSC.3,HSC.LTHSC.4,HSC.STHSC.1,HSC.STHSC.2,HSC.MPP.1,HSC.MPP.2,HSC.MPP.3,HSC.MPP.4,HSC.MPP.5
gene,,,,,,,,,,,,,,,,,,,,,
Il10,0,0,2,2,3181,1389,2988,2779,2,1,...,0,0,0,0,0,0,0,277,0,0
Pth1r,0,0,46,0,0,0,0,0,0,1,...,6,4,5,6,0,2,4,9,3,14
Fpr1,0,0,4,0,1773,0,2630,967,1,0,...,0,118,75,0,0,0,0,0,109,41
Erbb3,74,48,65,68,85,119,99,110,52,166,...,133,127,113,108,151,116,84,79,83,109
Gh,33,1,30,65,33,2,2,1,1,56,...,30,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Adm2,0,0,0,0,0,1,1,0,1,1,...,22,0,22,0,0,0,0,0,0,0
Ntsr1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0
Fbn1,1,0,0,0,0,0,0,0,0,0,...,18,5,64,5,0,10,31,5,13,90


## get score

In [23]:
def cal_power(a,b):
    return a*b/1000
def get_all_pair(exp1,exp2):
    col1 = list(exp1.columns)
    col2 = list(exp2.columns)
    
    df = pd.DataFrame()
    for i in col1:
        for j in col2:
            if i != j:
                p = cal_power (np.array(exp1[i]),  np.array(exp2[j]))
                df = pd.concat([df, pd.DataFrame(p, columns=[i+'_vs_'+j]) ] ,axis=1)
    return df

In [22]:
input_dir = '/home/UTHSCSA/hef/2.Project/z.other/2.Jack_lab/tsne_compare/specScore_top10'
exp = pd.read_table(input_dir + '/exp.txt', index_col=0)
exp

,B.nil.1,B.nil.2,B.nil.3,B.nil.4,B.LPS.1,B.LPS.2,B.LPS.3,B.LPS.4,B.CD154.1,B.CD154.2,...,HSC.LTHSC.2,HSC.LTHSC.3,HSC.LTHSC.4,HSC.STHSC.1,HSC.STHSC.2,HSC.MPP.1,HSC.MPP.2,HSC.MPP.3,HSC.MPP.4,HSC.MPP.5
gene,,,,,,,,,,,,,,,,,,,,,
Avp,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ifngr1,6705,7809,8480,8122,6161,2339,3591,3383,4534,7399,...,3118,2602,2580,3166,3202,3774,3050,3177,3348,3234
Ramp1,3356,0,2722,2500,1879,0,0,0,0,0,...,1050,1402,1019,2849,2598,1269,3947,3476,5134,3580
Il5,0,0,0,0,0,0,0,0,0,0,...,12,8,29,16,21,0,0,0,14,0
Glp2r,0,0,0,0,0,0,0,0,0,0,...,0,14,0,0,0,0,2,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cxcl10,13,0,2,1,2679,68,44,48,49,28,...,219,196,214,173,113,377,251,0,240,194
Tnfsf11,0,0,17,18,121,47,3,3,674,4,...,114,77,46,46,0,68,96,0,105,70
Esr1,410,345,397,303,65,1,54,54,86,176,...,515,362,425,761,812,208,676,710,708,816


In [ ]:
#will take a long computational time 
lst = [i for i in exp.index if (i in list(pair.GeneA))|(i in list(pair.GeneB))]
pair = pair.set_index('GeneA').T[[i for i in lst if i in list(pair.GeneA)]].T.reset_index().set_index('GeneB').T[[i for i in lst if i in list(pair.GeneB)]].T.reset_index()

id_a = list(pair.GeneA)
id_b = list(pair.GeneB)
exp_a = exp.T[id_a].T 
exp_b = exp.T[id_b].T 

power = get_all_pair(exp_a, exp_b)
power.index = [ id_a[i] + '_' + id_b[i] for i in range(len(pair)) ]
power.to_csv(input_dir + '/allpair_power.txt')
power 

# enrichment score

In [13]:
indir = '/home/UTHSCSA/hef/2.Project/z.other/2.Jack_lab/enrichment_FC'
id_name = pd.read_table(indir + '/id_rename.txt', sep = '\t',header = None)
id_name

,0,1
0,B.nil.1,B.nil
1,B.nil.2,B.nil
2,B.nil.3,B.nil
3,B.nil.4,B.nil
4,B.LPS.1,B.LPS
...,...,...
220,HSC.MPP.1,HSC.MPP
221,HSC.MPP.2,HSC.MPP
222,HSC.MPP.3,HSC.MPP
223,HSC.MPP.4,HSC.MPP


In [12]:
power = pd.read_table(indir + '/allpair_power.txt', sep = ',', index_col=0)
LRI = pd.read_table(indir + '/494_list.txt', index_col=0)
power = power.T[list(LRI.index)]
power 

,Adipoq_Adipor1,Adipoq_Adipor2,Adm_Calcrl,Adm_Ramp2,Adm_Ramp3,Adm2_Calcrl,Adm2_Ramp3,Angpt1_Tek,Angpt1_Tie1,Anxa1_Fpr1,...,Wnt3_Lrp6,Wnt3_Ryk,Wnt5b_Fzd1,Wnt5b_Fzd6,Wnt5b_Fzd7,Wnt8b_Fzd1,Wnt8b_Fzd2,Wnt8b_Lrp5,Wnt8b_Lrp6,Xcl1_Xcr1
B.nil.1_vs_B.nil.2,0.000,0.000,4.505,0.00,0.010,0.0,0.0,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B.nil.1_vs_B.nil.3,0.000,0.000,4.820,0.00,0.000,0.0,0.0,0.000,0.000,14.920,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B.nil.1_vs_B.nil.4,0.000,0.000,2.460,0.00,0.000,0.0,0.0,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B.nil.1_vs_B.LPS.1,0.000,0.000,2.610,0.00,6.465,0.0,0.0,0.000,0.000,6613.290,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B.nil.1_vs_B.LPS.2,0.000,0.000,1.825,0.18,0.000,0.0,0.0,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HSC.MPP.5_vs_HSC.STHSC.2,42.858,21.420,0.000,0.00,0.000,0.0,0.0,1030.226,5410.758,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HSC.MPP.5_vs_HSC.MPP.1,39.348,21.852,0.000,0.00,0.000,0.0,0.0,1764.918,3336.496,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HSC.MPP.5_vs_HSC.MPP.2,41.508,22.338,0.000,0.00,0.000,0.0,0.0,256.866,2245.506,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HSC.MPP.5_vs_HSC.MPP.3,42.714,19.062,0.000,0.00,0.000,0.0,0.0,353.536,1963.782,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
a = power
a['Ligand'] = a.index.str.split('_vs_').str[0]
a['Receptor'] = a.index.str.split('_vs_').str[1] 
a['Group_L'] = a['Ligand'].map(dict(id_name.values))
a['Group_R'] = a['Receptor'].map(dict(id_name.values))
a = a.groupby(['Group_L']).mean() 
a = a/a.mean() 
a

,Adipoq_Adipor1,Adipoq_Adipor2,Adm_Calcrl,Adm_Ramp2,Adm_Ramp3,Adm2_Calcrl,Adm2_Ramp3,Angpt1_Tek,Angpt1_Tie1,Anxa1_Fpr1,...,Wnt3_Lrp6,Wnt3_Ryk,Wnt5b_Fzd1,Wnt5b_Fzd6,Wnt5b_Fzd7,Wnt8b_Fzd1,Wnt8b_Fzd2,Wnt8b_Lrp5,Wnt8b_Lrp6,Xcl1_Xcr1
Group_L,,,,,,,,,,,,,,,,,,,,,
B.CD154,3.979758,3.979762,3.329812,3.374336,3.337340,0.042195,0.042185,0.000670,0.000676,1.160285,...,0.074487,0.074659,0.033446,0.032719,0.032574,0.000000,0.000000,0.000000,0.000000,3.605325
B.Fo,0.852137,0.853236,1.279475,1.298877,1.280950,0.000000,0.000000,0.023598,0.023793,0.352959,...,0.357390,0.358762,0.107003,0.104643,0.104347,0.682065,0.681821,0.666623,0.667122,0.068709
B.GC,1.859509,1.859141,0.703329,0.710422,0.702542,0.000000,0.000000,0.014748,0.014870,0.186777,...,0.521778,0.520061,0.468135,0.457757,0.456518,0.248670,0.248581,0.243267,0.243163,0.000000
B.LC/21,0.359937,0.359772,0.374479,0.378178,0.375365,1.012973,1.010020,0.187392,0.188925,0.297331,...,0.018630,0.018546,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026359
B.LPS,0.000000,0.000000,0.052079,0.052718,0.051743,0.042302,0.042189,0.000000,0.000000,0.248264,...,2.420183,2.425106,0.066872,0.065268,0.065093,0.000000,0.000000,0.000000,0.000000,0.955096
B.MZ/B1b,0.563566,0.564770,1.671996,1.698051,1.674463,0.000000,0.000000,0.026818,0.027038,0.482001,...,0.521260,0.521870,0.000000,0.000000,0.000000,0.053286,0.053267,0.052084,0.052067,0.050554
B.PC,0.438812,0.438085,0.926177,0.939092,0.925910,0.000000,0.000000,0.100568,0.101339,0.203828,...,0.783315,0.780702,0.234122,0.227608,0.226324,0.000000,0.000000,0.000000,0.000000,0.263723
B.Trans,1.535744,1.537699,1.692094,1.717265,1.693590,0.000000,0.000000,0.063020,0.063538,0.388726,...,0.856486,0.859700,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B.dev,0.219471,0.219542,0.892477,0.902681,0.892759,0.000000,0.000000,0.638251,0.643444,0.615930,...,0.000000,0.000000,0.284291,0.276436,0.276965,0.039965,0.039950,0.038831,0.039048,0.001100


In [15]:
FC = power/power.mean() 

FC['Ligand'] = FC.index.str.split('_vs_').str[0]
FC['Receptor'] = FC.index.str.split('_vs_').str[1]
FC['Group_L'] = FC['Ligand'].map(dict(id_name.values))
FC['Group_R'] = FC['Receptor'].map(dict(id_name.values))
FC = FC.groupby(['Group_L','Group_R']).mean().reset_index() 
FC.to_csv(indir + '/Relative_ES.csv', index=None)
FC 

/home/UTHSCSA/hef/.tmp/ipykernel_50564/3177950058.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  FC = power/power.mean()


,Group_L,Group_R,Adipoq_Adipor1,Adipoq_Adipor2,Adm2_Calcrl,Adm2_Ramp3,Adm_Calcrl,Adm_Ramp2,Adm_Ramp3,Angpt1_Tek,...,Wnt3_Lrp6,Wnt3_Ryk,Wnt5b_Fzd1,Wnt5b_Fzd6,Wnt5b_Fzd7,Wnt8b_Fzd1,Wnt8b_Fzd2,Wnt8b_Lrp5,Wnt8b_Lrp6,Xcl1_Xcr1
0,B.CD154,B.CD154,4.195869,4.447717,0.074937,0.000589,5.176326,4.220954,0.066404,0.000000,...,0.085433,0.027766,0.000314,0.007031,0.000296,0.000000,0.000000,0.000000,0.000000,6.929807
1,B.CD154,B.Fo,4.106648,3.196950,0.053805,0.000118,3.511707,0.000000,0.007716,0.000012,...,0.095459,0.028877,0.000566,0.009053,0.000000,0.000000,0.000000,0.000000,0.000000,0.231492
2,B.CD154,B.GC,3.823961,4.561733,0.018407,0.007360,1.201373,1.927551,0.482281,0.000006,...,0.080537,0.151418,0.001415,0.008862,0.000000,0.000000,0.000000,0.000000,0.000000,0.894593
3,B.CD154,B.LC/21,5.604201,6.392123,0.064988,0.049754,4.241562,10.312397,3.260221,0.000000,...,0.094679,0.225461,0.000354,0.013458,0.075661,0.000000,0.000000,0.000000,0.000000,0.464255
4,B.CD154,B.LPS,3.687814,5.982660,0.042832,0.278947,2.795504,2.523339,18.278459,0.000000,...,0.088590,0.085520,0.000707,0.052735,0.006435,0.000000,0.000000,0.000000,0.000000,0.817217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111,T.gd,T.CD8.Teff,1.030289,0.836184,0.163614,0.000000,0.316528,0.182034,0.000000,0.006270,...,0.281190,0.627430,0.000000,0.000000,0.000000,0.000000,1.600590,1.381252,1.225749,1.144275
2112,T.gd,T.CD8.Tmp,1.164816,0.779504,0.141395,0.016170,0.273543,0.165732,0.031406,0.000000,...,0.276194,0.424438,0.000000,0.000000,0.000000,0.000000,0.000000,1.186709,1.203972,1.121616
2113,T.gd,T.CD8.Trm,0.972983,0.779504,0.144929,0.075458,0.280381,0.000000,0.146562,0.000000,...,0.267630,0.818120,0.000000,0.070990,0.283323,0.000000,0.000000,0.000000,1.166639,1.759842
2114,T.gd,T.dev,0.823512,0.955451,0.236331,0.003554,0.457207,0.310983,0.006902,0.003666,...,0.564356,0.097474,0.022879,0.285000,0.212689,0.174425,0.000000,3.310215,2.460113,0.259706
